# Using Markov Chains to Predict Stock-Market Behaviour

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import io, base64, os, json, re 
import pandas as pd
import numpy as np
import datetime
from random import randint

In [56]:
data1 = pd.read_csv('spx.csv')
data1['Date'] = pd.to_datetime(data1['Date'])
cutoff = '2010-01-01'
data1 = data1[data1['Date'] >= cutoff]
gspc_df = data1
gspc_df

,Date,Open,High,Low,Close,Volume
35581,2010-01-04,1116.56,1133.87,1116.56,1132.99,2.217444e+09
35582,2010-01-05,1132.66,1136.63,1129.66,1136.52,1.383900e+09
35583,2010-01-06,1135.71,1139.19,1133.95,1137.14,2.762589e+09
35584,2010-01-07,1136.27,1142.46,1131.32,1141.69,2.928156e+09
35585,2010-01-08,1140.52,1145.39,1136.22,1144.98,2.438661e+09
...,...,...,...,...,...,...
39098,2023-12-22,4753.92,4772.94,4736.77,4754.63,1.743793e+09
39099,2023-12-26,4758.86,4784.72,4758.45,4774.75,1.447991e+09
39100,2023-12-27,4773.45,4785.39,4768.90,4781.58,1.611193e+09
39101,2023-12-28,4786.44,4793.30,4780.98,4783.35,1.481138e+09


In [57]:
new_set = []
for row_set in range(0, 100000):
    if row_set%2000 == 0:
        print(row_set)
    row_quant = randint(10, 30)
    row_start = randint(0, len(gspc_df) - row_quant)
    market_subset = gspc_df.iloc[row_start: row_start + row_quant]
    Close_Date = max(market_subset['Date'])
    if row_set%2000 == 0:
        print(Close_Date)
    Close_Gap = market_subset["Close"].pct_change()
    High_Gap = market_subset["High"].pct_change()
    Low_Gap = market_subset["Low"].pct_change()
    Volume_Gap = market_subset['Volume'].pct_change()
    Daily_Change = (market_subset["Close"] - market_subset["Open"])/ market_subset["Open"]
    Outcome_Next_Day_Direction = (market_subset["Volume"].shift(-1) - market_subset["Volume"])
    new_set.append(pd.DataFrame({'Sequence_ID':[row_set]*len(market_subset),
                            'Close_Date':[Close_Date]*len(market_subset),
                           'Close_Gap':Close_Gap,
                           'High_Gap':High_Gap,
                           'Low_Gap':Low_Gap,
                           'Volume_Gap':Volume_Gap,
                           'Daily_Change':Daily_Change,
                           'Outcome_Next_Day_Direction':Outcome_Next_Day_Direction}))                                                                            

0
2021-08-23 00:00:00
2000
2012-03-01 00:00:00
4000
2017-02-09 00:00:00
6000
2020-03-13 00:00:00
8000
2013-10-28 00:00:00
10000
2010-07-13 00:00:00
12000
2014-10-16 00:00:00
14000
2017-07-26 00:00:00
16000
2023-08-25 00:00:00
18000
2020-01-29 00:00:00
20000
2015-03-31 00:00:00
22000
2011-06-30 00:00:00
24000
2017-07-20 00:00:00
26000
2020-04-03 00:00:00
28000
2012-01-06 00:00:00
30000
2021-09-09 00:00:00
32000
2011-08-04 00:00:00
34000
2018-06-11 00:00:00
36000
2011-07-07 00:00:00
38000
2018-02-23 00:00:00
40000
2019-12-30 00:00:00
42000
2016-09-22 00:00:00
44000
2017-05-31 00:00:00
46000
2019-11-14 00:00:00
48000
2011-08-30 00:00:00
50000
2010-03-18 00:00:00
52000
2019-04-03 00:00:00
54000
2011-06-28 00:00:00
56000
2020-07-01 00:00:00
58000
2016-11-10 00:00:00
60000
2018-07-23 00:00:00
62000
2023-07-10 00:00:00
64000
2010-05-10 00:00:00
66000
2012-03-23 00:00:00
68000
2015-10-23 00:00:00
70000
2012-08-24 00:00:00
72000
2023-10-25 00:00:00
74000
2010-12-30 00:00:00
76000
2019-05-16 00:

In [58]:
len(market_subset)

20

In [59]:
new_set_df = pd.concat(new_set)
print(new_set_df.shape)
new_set_df = new_set_df.dropna(how = 'any')
print(new_set_df.shape)
new_set_df.tail(20)

(2002558, 8)
(1802558, 8)


,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
38383,99998,2021-02-24,-0.007733,-0.006994,-0.007266,0.188760,-0.002329,194822630.0
38384,99998,2021-02-24,0.001256,-0.001778,-0.017839,0.070773,0.006300,-159452439.0
38062,99999,2019-12-06,-0.001962,-0.001539,0.000729,-0.132495,0.002169,239155555.0
38063,99999,2019-12-06,0.001565,0.004624,0.002897,0.141814,0.000829,24038889.0
38064,99999,2019-12-06,0.000712,-0.001467,-0.001922,0.012484,0.003197,-129561111.0
38065,99999,2019-12-06,0.000837,0.000045,0.001449,-0.066455,0.001902,33100000.0
38066,99999,2019-12-06,0.007695,0.007185,0.006921,0.018186,0.004035,56133333.0
38067,99999,2019-12-06,0.000503,0.001189,0.002403,0.030291,0.001321,85211111.0
38068,99999,2019-12-06,-0.000593,0.001111,0.000453,0.044630,-0.002325,247122223.0
38069,99999,2019-12-06,-0.003756,-0.002772,-0.007085,0.123903,-0.001991,-174627778.0


In [60]:
new_set_df.head()

,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
38489,0,2021-08-23,0.010145,0.010369,0.007159,-0.050418,0.006982,59856678.0
38490,0,2021-08-23,0.002382,0.001710,0.005535,0.034157,0.002885,273393295.0
38491,0,2021-08-23,-0.004712,-0.001436,-0.007477,0.150859,-0.003378,33203527.0
38492,0,2021-08-23,-0.000186,-0.000206,0.003316,0.015920,-0.000525,-99489130.0
38493,0,2021-08-23,0.004206,0.003284,0.003779,-0.046954,0.003533,226492094.0


Create the sequences and simplify the data by binning into groups

In [61]:
#Close Gap 
new_set_df['Close_Gap_LMH'] = pd.qcut(new_set_df['Close_Gap'], 3, labels=["L", "M", "H"])

# High_Gap
new_set_df['High_Gap_LMH'] = pd.qcut(new_set_df['High_Gap'], 3, labels=["L", "M", "H"])

# Low_Gap
new_set_df['Low_Gap_LMH'] = pd.qcut(new_set_df['Low_Gap'], 3, labels=["L", "M", "H"])

#Volume Gap
new_set_df['Volume_Gap_LMH'] = pd.qcut(new_set_df['Volume_Gap'], 3, labels=["L", "M", "H"])

#Daily_Change
new_set_df['Daily_Change_LMH'] = pd.qcut(new_set_df['Daily_Change'], 3, labels=["L", "M", "H"])

new_set_df['Event_Pattern'] = new_set_df['Close_Gap_LMH'].astype(str) + new_set_df['Volume_Gap_LMH'].astype(str) + new_set_df['Daily_Change_LMH'].astype(str)

In [62]:
new_set_df.tail(10)

,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction,Close_Gap_LMH,High_Gap_LMH,Low_Gap_LMH,Volume_Gap_LMH,Daily_Change_LMH,Event_Pattern
38070,99999,2019-12-06,-0.001583,-0.002841,0.001016,-0.077903,-0.001592,-274322222.0,M,L,M,L,M,MLM
38071,99999,2019-12-06,0.002175,0.000887,0.001522,-0.132717,-0.000360,158194444.0,M,M,M,L,M,MLM
38072,99999,2019-12-06,0.007507,0.006733,0.005866,0.088246,0.005197,602255556.0,H,H,H,H,H,HHH
38073,99999,2019-12-06,0.002196,0.002827,0.004350,0.308714,0.001809,-868055556.0,M,M,H,H,M,MHM
38074,99999,2019-12-06,0.004174,0.003682,0.003964,-0.340000,0.002588,-716705556.0,H,H,H,L,M,HLM
38075,99999,2019-12-06,-0.004011,-0.001255,-0.001295,-0.425332,-0.001970,847622223.0,L,M,M,L,L,LLL
38076,99999,2019-12-06,-0.008631,-0.001901,-0.009097,0.875331,-0.009536,213694444.0,L,L,L,H,L,LHL
38077,99999,2019-12-06,-0.006638,-0.015692,-0.013003,0.117675,0.001875,23133333.0,L,L,L,H,M,LHM
38078,99999,2019-12-06,0.006324,0.007887,0.010487,0.011398,0.002984,-188488888.0,H,H,H,M,M,HMM
38079,99999,2019-12-06,0.001500,0.000022,0.000396,-0.091821,-0.000571,68738888.0,M,M,M,L,M,MLM


In [63]:
new_set_df['Outcome_Next_Day_Direction'].describe()

count    1.802558e+06
mean    -1.298416e+05
std      4.285100e+08
min     -2.877287e+09
25%     -1.707292e+08
50%     -6.711111e+06
75%      1.773866e+08
max      2.894422e+09
Name: Outcome_Next_Day_Direction, dtype: float64

In [64]:
# group the set
compressed_set = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index()
print(compressed_set.shape)
compressed_set.head()

(100000, 3)


,Sequence_ID,Close_Date,Event_Pattern
0,0,2021-08-23,"{HMH, MMM, LHL, MMM, HMH, LHM, MLL, HHH, LHL, ..."
1,1,2013-02-28,"{HLH, MML, HHH, LLL, LHL, HMH, LLL, HHH, MMM, ..."
2,2,2016-04-04,"{HLH, MLM, MMM, HHH, HHH, HHH, MLM, MMM, LHL, ..."
3,3,2019-12-26,"{LHL, MLM, MLM, HHH, MHM, HLM, LLL, LHL, LHM, ..."
4,4,2016-11-16,"{LML, MMM, LHL, LLL, LLL, MMM, HMH, HMH, HHH, ..."


In [65]:
compressed_outcomes = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Outcome_Next_Day_Direction'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()
print(compressed_outcomes.shape)
compressed_outcomes.describe()

(100000, 3)


,Sequence_ID,Close_Date,Outcome_Next_Day_Direction
count,100000.000000,100000,1.000000e+05
mean,49999.500000,2017-01-16 03:11:09.600000,-1.147260e+05
min,0.000000,2010-01-15 00:00:00,-4.788479e+08
25%,24999.750000,2013-07-22 00:00:00,-1.673854e+07
50%,49999.500000,2017-01-26 00:00:00,-4.600693e+05
75%,74999.250000,2020-07-07 00:00:00,1.587989e+07
max,99999.000000,2023-12-29 00:00:00,3.948142e+08
std,28867.657797,NaN,4.067653e+07


In [66]:
compressed_set = pd.merge(compressed_set, compressed_outcomes, on=['Sequence_ID', 'Close_Date'], how = 'inner')
print(compressed_set.shape)
compressed_set.head()

(100000, 4)


,Sequence_ID,Close_Date,Event_Pattern,Outcome_Next_Day_Direction
0,0,2021-08-23,"{HMH, MMM, LHL, MMM, HMH, LHM, MLL, HHH, LHL, ...",4.865159e+06
1,1,2013-02-28,"{HLH, MML, HHH, LLL, LHL, HMH, LLL, HHH, MMM, ...",1.053213e+07
2,2,2016-04-04,"{HLH, MLM, MMM, HHH, HHH, HHH, MLM, MMM, LHL, ...",-2.195963e+07
3,3,2019-12-26,"{LHL, MLM, MLM, HHH, MHM, HLM, LLL, LHL, LHM, ...",-4.338449e+07
4,4,2016-11-16,"{LML, MMM, LHL, LLL, LLL, MMM, HMH, HMH, HHH, ...",-8.073504e+06


In [67]:
compressed_set['Event_Pattern'] = [''.join(e.split()).replace('{','').replace('}','') for e in compressed_set['Event_Pattern'].values]
compressed_set.head()

,Sequence_ID,Close_Date,Event_Pattern,Outcome_Next_Day_Direction
0,0,2021-08-23,"HMH,MMM,LHL,MMM,HMH,LHM,MLL,HHH,LHL,HLH,MLM,ML...",4.865159e+06
1,1,2013-02-28,"HLH,MML,HHH,LLL,LHL,HMH,LLL,HHH,MMM,MML,HLH,ML...",1.053213e+07
2,2,2016-04-04,"HLH,MLM,MMM,HHH,HHH,HHH,MLM,MMM,LHL,MLM,MLM,HH...",-2.195963e+07
3,3,2019-12-26,"LHL,MLM,MLM,HHH,MHM,HLM,LLL,LHL,LHM,HMM,MLM,HM...",-4.338449e+07
4,4,2016-11-16,"LML,MMM,LHL,LLL,LLL,MMM,HMH,HMH,HHH,MMM,MLM,MH...",-8.073504e+06


In [68]:
# validation
compressed_set_validation = compressed_set[compressed_set['Close_Date'] >= datetime.datetime.now() - datetime.timedelta(days = 90)]
compressed_set_validation.shape

(1744, 4)

In [69]:
compressed_set = compressed_set[compressed_set['Close_Date'] < datetime.datetime.now() - datetime.timedelta(days = 90)]  
compressed_set.shape

(98256, 4)

In [70]:
list(compressed_set)

['Sequence_ID', 'Close_Date', 'Event_Pattern', 'Outcome_Next_Day_Direction']

In [71]:
# don't need the date
compressed_set = compressed_set[['Sequence_ID', 'Event_Pattern','Outcome_Next_Day_Direction']]
compressed_set_validation = compressed_set_validation[['Sequence_ID', 'Event_Pattern','Outcome_Next_Day_Direction']]
compressed_set.head(), compressed_set_validation.head()

(   Sequence_ID                                      Event_Pattern  \
 0            0  HMH,MMM,LHL,MMM,HMH,LHM,MLL,HHH,LHL,HLH,MLM,ML...   
 1            1  HLH,MML,HHH,LLL,LHL,HMH,LLL,HHH,MMM,MML,HLH,ML...   
 2            2  HLH,MLM,MMM,HHH,HHH,HHH,MLM,MMM,LHL,MLM,MLM,HH...   
 3            3  LHL,MLM,MLM,HHH,MHM,HLM,LLL,LHL,LHM,HMM,MLM,HM...   
 4            4  LML,MMM,LHL,LLL,LLL,MMM,HMH,HMH,HHH,MMM,MLM,MH...   
 
    Outcome_Next_Day_Direction  
 0                4.865159e+06  
 1                1.053213e+07  
 2               -2.195963e+07  
 3               -4.338449e+07  
 4               -8.073504e+06  ,
      Sequence_ID                                      Event_Pattern  \
 59            59  LML,HMH,MLM,HHH,MMM,MMM,MLM,HMH,LLM,HLM,MLM,MH...   
 72            72  MLM,HMH,LLM,HLM,MLM,MHM,MMM,MHL,HHM,HLH,LMM,ML...   
 269          269  LML,HMH,MMM,HHH,HLH,HHH,HMM,LHL,LML,HLH,MHH,LM...   
 411          411  LMM,LML,LHL,LLL,HLH,LHL,MHL,HMH,LHL,MMM,LML,HM...   
 476          476  

In [72]:
compressed_set['Outcome_Next_Day_Direction'].describe()

count    9.825600e+04
mean    -1.544478e+05
std      4.045735e+07
min     -4.346970e+08
25%     -1.675714e+07
50%     -5.314814e+05
75%      1.576181e+07
max      3.948142e+08
Name: Outcome_Next_Day_Direction, dtype: float64

In [73]:
print(len(compressed_set['Outcome_Next_Day_Direction']))
len(compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000])

98256


64839

In [74]:
# remove the small/uninteresting moves
print('all moves: ', len(compressed_set))
compressed_set = compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000]
compressed_set['Outcome_Next_Day_Direction'] = np.where((compressed_set['Outcome_Next_Day_Direction'] > 0), 1, 0)
compressed_set_validation['Outcome_Next_Day_Direction'] = np.where((compressed_set_validation['Outcome_Next_Day_Direction'] > 0), 1, 0)
print('big moves only:', len(compressed_set))

all moves:  98256
big moves only: 64839


In [75]:
compressed_set.head()

,Sequence_ID,Event_Pattern,Outcome_Next_Day_Direction
1,1,"HLH,MML,HHH,LLL,LHL,HMH,LLL,HHH,MMM,MML,HLH,ML...",1
2,2,"HLH,MLM,MMM,HHH,HHH,HHH,MLM,MMM,LHL,MLM,MLM,HH...",0
3,3,"LHL,MLM,MLM,HHH,MHM,HLM,LLL,LHL,LHM,HMM,MLM,HM...",0
8,8,"MMM,MMM,LLL,HMM,MHM,MMM,MMM,MMM,MHM,MMM,MHL,HL...",1
11,11,"HMH,HLH,HLH,MHL,LLL,LHL,MLM,HMH,LML,HMH,MLM,LM...",0


In [76]:
compressed_set_validation.head()

,Sequence_ID,Event_Pattern,Outcome_Next_Day_Direction
59,59,"LML,HMH,MLM,HHH,MMM,MMM,MLM,HMH,LLM,HLM,MLM,MH...",1
72,72,"MLM,HMH,LLM,HLM,MLM,MHM,MMM,MHL,HHM,HLH,LMM,ML...",1
269,269,"LML,HMH,MMM,HHH,HLH,HHH,HMM,LHL,LML,HLH,MHH,LM...",1
411,411,"LMM,LML,LHL,LLL,HLH,LHL,MHL,HMH,LHL,MMM,LML,HM...",1
476,476,"HLH,MHH,LML,LHL,LML,MLM,HMM,LHL,LHL,LLL,HLH,HM...",0


In [77]:
# create two separate datasets
compressed_set_pos = compressed_set[compressed_set['Outcome_Next_Day_Direction'] == 1][['Sequence_ID', 'Event_Pattern']]
compressed_set_neg = compressed_set[compressed_set['Outcome_Next_Day_Direction'] == 0][['Sequence_ID', 'Event_Pattern']]
compressed_set_pos.shape, compressed_set_neg.shape

((31668, 2), (33171, 2))

In [95]:
flat_list = [item.split(',') for item in compressed_set['Event_Pattern'].values]
unique_patterns = ','.join(str(r) for v in flat_list for r in v)
unique_patterns = list(set(unique_patterns.split(',')))
len(unique_patterns)

27

In [96]:
compressed_set['Outcome_Next_Day_Direction'].head() 

1     1
2     0
3     0
8     1
11    0
Name: Outcome_Next_Day_Direction, dtype: int32

Building the Markov Chain

In [105]:
# build the markov transition grid
def build_transition_grid(compressed_grid, unique_patterns):
    # build the markov transition grid

    patterns = []
    counts = []
    for from_event in unique_patterns:

        # how many times 
        for to_event in unique_patterns:
            pattern = from_event + ',' + to_event # MMM,MlM

            ids_matches = compressed_grid[compressed_grid['Event_Pattern'].str.contains(pattern)]
            found = 0
            if len(ids_matches) > 0:
                Event_Pattern = '---'.join(ids_matches['Event_Pattern'].values)
                found = Event_Pattern.count(pattern)
            patterns.append(pattern)
            counts.append(found)

    # create to/from grid
    grid_Df = pd.DataFrame({'pairs':patterns, 'counts': counts})

    grid_Df[['x', 'y']] = grid_Df['pairs'].str.split(',', n=1, expand=True)
    grid_Df.head()

    grid_Df = grid_Df.pivot(index='x', columns='y', values='counts')

    grid_Df.columns= [col for col in grid_Df.columns]
    grid_Df.index.name = None

    # replace all NaN with zeros
    grid_Df.fillna(0, inplace=True)
    grid_Df.head()

    #grid_Df.rowSums(transition_dataframe) 
    grid_Df = grid_Df / grid_Df.sum(1)
    return (grid_Df)
 

In [106]:
grid_pos = build_transition_grid(compressed_set_pos, unique_patterns) 
grid_neg = build_transition_grid(compressed_set_neg, unique_patterns) 

In [107]:
grid_neg.head()

,HHH,HHL,HHM,HLH,HLL,HLM,HMH,HML,HMM,LHH,...,LMM,MHH,MHL,MHM,MLH,MLL,MLM,MMH,MML,MMM
HHH,0.049845,0.138389,0.021493,0.092648,0.000000,0.258423,0.069876,0.0,0.069922,0.000000,...,0.060148,0.000000,0.039390,0.027464,0.064717,0.168121,0.170917,0.059759,0.010392,0.088523
HHL,0.002348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002835,0.000000,0.000000,0.000000
HHM,0.004053,0.000000,0.006399,0.016795,0.241111,0.054795,0.013058,0.0,0.000000,0.000000,...,0.000000,0.000000,0.017476,0.012955,0.000000,0.024287,0.020816,0.000000,0.000000,0.017945
HLH,0.149657,0.255668,0.103199,0.084480,0.000000,0.087190,0.119943,0.0,0.138146,0.292194,...,0.099230,0.236564,0.152011,0.127371,0.222651,0.042757,0.100490,0.149894,0.249096,0.136922
HLL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012871,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003698,0.000000,0.000000,0.000000


In [108]:
grid_pos.head()

,HHH,HHL,HHM,HLH,HLL,HLM,HMH,HML,HMM,LHH,...,LMM,MHH,MHL,MHM,MLH,MLL,MLM,MMH,MML,MMM
HHH,0.056387,0.265537,0.030802,0.092564,0.000000,0.188511,0.085611,0.0,0.116505,0.000000,...,0.163768,0.00000,0.145223,0.044858,0.097039,0.096160,0.132941,0.096809,0.043521,0.104167
HHL,0.006194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.003619,0.000000,0.000000,0.000000
HHM,0.002526,0.000000,0.048043,0.016649,0.264516,0.015689,0.014385,0.0,0.000000,0.000000,...,0.000000,0.00000,0.016926,0.013432,0.000000,0.010717,0.019633,0.000000,0.000000,0.015234
HLH,0.134761,0.066384,0.060054,0.065473,0.000000,0.144340,0.118070,0.0,0.079733,0.237242,...,0.044384,0.15753,0.092849,0.102072,0.105958,0.091396,0.070120,0.107625,0.185088,0.100179
HLL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002357,0.0,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000950,0.000000,0.000000,0.000000


In [109]:
def log(x,y):
    try:
        lg = np.log(x/y)
    except:
        lg = 0
    return lg
actual = []
predicted = []
for seq_id in compressed_set_validation['Sequence_ID'].values:
    patterns = compressed_set_validation[compressed_set_validation['Sequence_ID'] == seq_id]['Event_Pattern'].values[0].split(',')
    pos = []
    neg = []
    log_odds = []
    for id in range(0, len(patterns)-1):
        if (patterns[id] in list(grid_pos) and patterns[id+1] in list(grid_pos) and patterns[id] in list(grid_neg) and patterns[id+1] in list(grid_neg)):
            numerator = grid_pos[patterns[id]][patterns[id+1]]
            denominator = grid_neg[patterns[id]][patterns[id+1]]
            if (numerator == 0 and denominator == 0):
                log_value =0
            elif (denominator == 0):
                log_value = np.log(numerator / 0.00001)
            elif (numerator == 0):
                log_value = np.log(0.00001 / denominator)
            else:
                log_value = np.log(numerator/denominator)
        else:
            log_value = 0
        
        log_odds.append(log_value)
        
        pos.append(numerator)
        neg.append(denominator)
      
    print('outcome:', compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    print(sum(pos)/sum(neg))
    print(sum(log_odds))

    actual.append(compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    predicted.append(sum(log_odds))

from sklearn.metrics import confusion_matrix

confusion_matrix(actual, [1 if p > 0 else 0 for p in predicted])

outcome: 1
0.96832265726175
-1.6785645702372016
outcome: 1
1.0262076715814379
-0.2792647074091004
outcome: 1
1.0590421827875944
1.9337050835348726
outcome: 1
1.0065469118835921
1.0146215594561274
outcome: 0
0.9556112016957531
0.15814095419243346
outcome: 1
1.1247361815937875
0.4580115792991464
outcome: 1
0.9808731778539861
0.6651137706089045
outcome: 0
1.1323664293201692
1.0765704118470762
outcome: 0
0.9931444619517334
0.2915843790985654
outcome: 0
0.9865209026497648
-0.6767128746213091
outcome: 1
0.958240103130703
-1.547273454127301
outcome: 0
0.9213767176745631
-2.094114001121161
outcome: 0
0.9715098111199904
-0.2188073209241675
outcome: 1
0.9844773310439766
0.33946406725317146
outcome: 1
1.0057213230579565
1.3871207407831705
outcome: 1
1.012130701003245
1.6090724115423263
outcome: 0
0.9640083859234236
-2.1449479747577254
outcome: 1
1.0184015923471856
1.2887038355189175
outcome: 1
0.9940351975898853
0.567749641144486
outcome: 0
0.9784912517740787
-1.5125600322168034
outcome: 0
1.0099

outcome: 1
0.9869069459614832
-1.0604241457450438
outcome: 1
0.9999586583621636
0.8717638055987607
outcome: 0
1.000890663049904
1.0302746483500043
outcome: 1
0.9811453099727931
-1.285432356853314
outcome: 0
0.9380164103383626
-0.688128551647756
outcome: 1
1.1247361815937875
0.4580115792991464
outcome: 1
1.0110488042210133
1.3391802934339023
outcome: 0
0.933006007214291
-0.6723891451811841
outcome: 0
0.9595522117957002
-0.02437958267640894
outcome: 1
0.9879511352624343
0.6431288388120917
outcome: 1
1.0762280782149818
2.11797143823666
outcome: 0
0.9905072977886649
-0.11772071513766588
outcome: 1
1.0280623389633425
1.6557682672099387
outcome: 1
0.9868530425731166
-1.275775821363486
outcome: 1
1.076765484158154
1.8988672141348422
outcome: 0
0.953367342245014
0.1043619186219295
outcome: 1
0.9834110522806072
-1.1052420716883058
outcome: 1
0.9962586465583557
0.3227532508157942
outcome: 1
0.9876286801735872
0.5856100608827832
outcome: 1
0.9974562458968345
1.7077196724242016
outcome: 1
1.051938

outcome: 0
0.9836891282024064
-0.8970304596552972
outcome: 0
0.9692326931664135
-1.887289728105125
outcome: 1
0.9994194527490807
1.0550792947812575
outcome: 1
0.9913725511731302
0.6006861508039124
outcome: 0
1.0310697493617103
1.145745364009184
outcome: 1
1.018899859540722
2.0610062979067205
outcome: 0
0.9426047710999045
-0.5641725377923763
outcome: 0
1.0231748126958782
0.6578924923967008
outcome: 0
0.9048457521951988
-2.614070887463204
outcome: 0
1.0370354754125402
0.4586458376917343
outcome: 0
0.9811698010813604
-1.365056140949279
outcome: 0
1.0010637372125002
-0.9239837568912376
outcome: 1
0.966324783586277
-1.9634371314266197
outcome: 1
1.105102767234183
1.327101718559475
outcome: 0
0.9356806285443128
-1.8567963616469818
outcome: 1
0.9385363914591948
-1.6410853532689582
outcome: 0
0.9601709194474581
-0.6163432103577363
outcome: 1
0.992996922737106
1.0851411633487515
outcome: 1
1.0110488042210133
1.3391802934339023
outcome: 1
0.9772703838501196
-1.1185698074677524
outcome: 1
1.02955

outcome: 1
1.0128781855591673
1.108780415715355
outcome: 1
0.9936259598094777
0.46679966793637206
outcome: 0
1.0239371865677855
-0.21630326320477894
outcome: 0
1.0592625484396299
-0.3997004817720762
outcome: 0
0.9549208334335622
-1.964815789868311
outcome: 1
1.0008606479868372
1.0170117922967372
outcome: 0
0.9966789340814015
0.29872529343009485
outcome: 1
0.974478188065629
-1.4715609606493623
outcome: 1
0.9498423725857431
0.1484670164965711
outcome: 0
0.9993514717143881
-0.9745228367058694
outcome: 0
1.0330289772540195
0.7884798612929593
outcome: 0
1.0138604728246234
1.2093593363327926
outcome: 0
1.0047788354803413
-0.9987692362012676
outcome: 1
1.0519382893200304
0.48572674093758217
outcome: 0
0.9843191840378084
-0.21957992415149044
outcome: 1
0.9671923020981997
-1.2208019648536295
outcome: 1
0.9498423725857431
0.1484670164965711
outcome: 1
1.0144792918013896
1.8966697788782576
outcome: 0
1.0114608372330862
0.5420409142440017
outcome: 0
0.951216709116447
-1.5648356301282804
outcome: 1

outcome: 1
0.9833157120415293
0.44797674894362394
outcome: 1
1.0192146341522472
0.8635198731456329
outcome: 1
0.9785757489647052
0.9621803296742116
outcome: 1
1.0025785457756837
-0.7104233633300157
outcome: 0
0.9601709194474581
-0.6163432103577363
outcome: 1
0.9950412612480776
-0.6576225429642407
outcome: 0
0.9406363540203446
-0.530576557941553
outcome: 0
0.9490807822295877
-2.1560593081640578
outcome: 1
1.0570368664942411
0.6213770156673035
outcome: 1
1.0208112933346067
1.2214738938922935
outcome: 1
1.0192146341522472
0.8635198731456329
outcome: 0
1.0282009204267735
1.7246828497651823
outcome: 1
1.0104481236667824
0.614824663893937
outcome: 0
1.0098272994869748
0.7310965730983158
outcome: 1
0.9485737044051697
-0.09594549985429368
outcome: 1
1.0048558617942935
0.3892770841355565
outcome: 0
0.9977911640756454
-1.3567131701635722
outcome: 0
0.9781185198769389
-1.6382718056772163
outcome: 0
0.966018961365385
-0.2546907105313825
outcome: 0
0.9786656313434048
0.16424877841536475
outcome: 1


outcome: 0
0.9777483799692304
-1.8251355615562985
outcome: 1
1.0212440822268674
1.4341663828269295
outcome: 0
0.9350560984129127
-1.081044041665649
outcome: 1
1.1268759164256363
1.243253098243031
outcome: 0
0.9685154647156412
0.13759427376714822
outcome: 1
1.0154867055662886
0.8867589195575165
outcome: 1
1.011254038126779
2.187482900196636
outcome: 0
0.9453866437108571
0.18178466617865494
outcome: 1
1.0647003351342244
2.089929774760034
outcome: 0
1.0252839934531326
0.8865167071161173
outcome: 0
0.9462993242846631
-1.595532573661233
outcome: 0
1.1072268906848914
1.4513891069150013
outcome: 1
0.9214970088065572
-0.8385911641651096
outcome: 1
0.986089981544025
-1.1037045205546716
outcome: 0
0.9677660660996641
-1.4943915934013279
outcome: 1
1.0106586664391481
-1.1471371954784788
outcome: 1
0.9641450496061456
0.393421727386724
outcome: 0
1.0250221290685024
1.5976094806028156
outcome: 0
0.9254638544978552
-0.6210629275089137
outcome: 1
0.9788022349066856
0.4215269544290824
outcome: 0
0.92893

outcome: 0
0.9806230332269957
-0.9605563277701015
outcome: 1
1.0333850508120501
-0.07087460775001195
outcome: 1
1.040363012827501
-0.821305621653102
outcome: 1
1.0836588627001993
1.2430012667538044
outcome: 1
0.9585130409732708
0.3660510208804425
outcome: 0
0.9293450490381507
-1.5632536509874522
outcome: 1
0.9741151425601499
-1.5490145274951252
outcome: 0
1.0188031325193387
0.5654599869212525
outcome: 1
0.9875783435248163
0.8853855073403121
outcome: 0
0.8645615639366719
-1.6852584894590297
outcome: 0
0.9556112016957531
0.15814095419243346
outcome: 0
0.974550492441197
-1.742000278896477
outcome: 1
1.0015119741252432
0.74113378631179
outcome: 0
0.9467092393824268
-0.39578692076038047
outcome: 1
1.0297901374886167
-0.23124247219605554
outcome: 0
1.0145370814452417
1.1949408762686617
outcome: 1
1.0040411088639438
-1.034624733422445
outcome: 1
0.9585130409732708
0.3660510208804425
outcome: 1
1.023495892301135
1.903569276952607
outcome: 0
0.9356806285443128
-1.8567963616469818
outcome: 1
0.9

outcome: 0
0.9843137980270387
0.596924313648036
outcome: 0
0.9791647662209514
0.340903455465052
outcome: 0
0.9198672830180821
-1.2453677550639717
outcome: 1
1.0040433377759506
-1.0341612678318157
outcome: 1
0.9575205016294221
0.0999053394857491
outcome: 1
0.9311306455658436
-2.0777251002078847
outcome: 1
0.9778462820354749
-1.4754805475912776
outcome: 1
0.9747384777412639
0.8305300993736173
outcome: 1
0.9305657169294573
-1.9675568425426297
outcome: 0
0.9437652338222648
-0.3429129553397715
outcome: 0
1.0048962518232947
-0.979769032216374
outcome: 1
1.0134749852441518
2.082172100031444
outcome: 0
0.9508315113673084
-0.9887444097968188
outcome: 0
0.9198672830180821
-1.2453677550639717
outcome: 1
0.9867310310352386
-0.6889992069938706
outcome: 0
1.0515426047414542
0.7596725579336064
outcome: 1
0.9849323468248476
0.8867156549919137
outcome: 1
0.9912941681155109
0.7857884614288804
outcome: 1
0.9973322027339455
1.5963054665988075
outcome: 1
0.9983892471955732
-0.12678617417885069
outcome: 0
0

outcome: 1
1.012997095486941
0.4678443016587371
outcome: 0
1.036557350883755
1.0945572397391137
outcome: 1
1.0174955990346208
0.6514388842347976
outcome: 0
1.1072268906848914
1.4513891069150013
outcome: 0
0.9307567185473161
-2.2222222050316844
outcome: 1
1.0232452400665195
1.158870574554526
outcome: 0
0.9933009818119292
-1.345875430520689
outcome: 1
1.0127298132992906
0.3225519023317567
outcome: 0
1.0241720404060337
-0.38377185025113886
outcome: 0
0.9688671870480144
0.37605396184411005
outcome: 0
0.9565639100975377
0.10821660738880763
outcome: 1
1.0226716378773653
2.2826081822897297
outcome: 1
1.0128429317012955
0.6478426731216591
outcome: 1
1.0151986328943887
1.4125013995092568
outcome: 0
1.0082444061857785
0.22290160709801476
outcome: 0
1.0129227906817948
1.122043271768622
outcome: 0
0.9416858879522744
-0.941981709768963
outcome: 1
1.0134110552102218
0.9998720095092842
outcome: 0
0.9554743347437337
-0.6326761452462291
outcome: 0
0.9406363540203446
-0.530576557941553
outcome: 0
1.0236

array([[500, 275],
       [272, 697]], dtype=int64)

In [110]:
from sklearn.metrics import accuracy_score
score = accuracy_score(actual, [1 if p > 0 else 0 for p in predicted])
print('Accuracy:', round(score * 100,2), '%')

Accuracy: 68.64 %
